# Track

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import sys
import os

import numpy as np

import unittest

from file_reader import getFileReader

## Main Class

In [2]:
class Track():
    '''GPS track class - reading, analysing, writing, etc.'''

    def __init__(self, filename):
        '''Basic init just records the filename'''

        self.filename = filename


    def load(self):
        '''Load the track into memory'''

        self.reader = getFileReader(self.filename)
        self.reader.load()

        self.data = self.reader.data
        self.numRecords = self.reader.numRecords

        self.fields = {fieldName: self.getFieldInfo(array) for fieldName, array in self.data.items()}

        
    def getFieldInfo(self, array):
        '''Determine the number of decimal places being used by the field'''
        
        fieldInfo = {'type': array.dtype.name,
                     'decimals': self.getDecimals(array)}
       
        return fieldInfo


    def getDecimals(self, array, maxDecDigits=None):
        '''Determine the number of decimal places being used in the array'''
        
        # maxDecDigits is currently unused but is available for the future
        if maxDecDigits is None:
            maxAbs = np.max(np.abs(array))
            if maxAbs > 0:
                maxIntDigits = int(np.floor(np.log10(maxAbs)) + 1)
            else:
                maxIntDigits = 0

            if array.dtype.name == 'float64':
                maxDecDigits = 15 - maxIntDigits
            elif array.dtype.name == 'float32':
                maxDecDigits = 6 - maxIntDigits
            elif array.dtype.name == 'float16':
                maxDecDigits = 3 - maxIntDigits
            else:
                return 0
        
        # Scale the float so that it can be treated as an integer
        scale = 10 ** maxDecDigits
        scaled = np.round(array * scale)

        divisor = 10
        decimals = maxDecDigits

        while divisor <= scale and not np.any(scaled % divisor):
            divisor *= 10
            decimals -= 1

        return decimals

## Unit Tests

In [3]:
class TestReaders(unittest.TestCase):
    '''Class to test all of the file readers'''

    def testFitApexPro(self):
        '''Test loading of an FIT file from COROS Apex Pro'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'APEX_Pro_Speedsurfing20220411111317.fit')
        track = Track(filename)
        track.load()
        
        self.assertEqual(track.fields['ts']['decimals'], 0)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['dist']['decimals'], 1)
        self.assertEqual(track.fields['sog']['decimals'], 3)
        self.assertEqual(track.fields['cog']['decimals'], 0)
        self.assertEqual(track.fields['hdop']['decimals'], 1)


    def testGpxApexPro(self):
        '''Test loading of an GPX file from COROS Apex Pro'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'APEX_Pro_Speedsurfing20220411111317.gpx')
        track = Track(filename)
        track.load()

        self.assertEqual(track.fields['ts']['decimals'], 0)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['cog']['decimals'], 0)
        self.assertEqual(track.fields['hdop']['decimals'], 1)


    def testGpxSbn(self):
        '''Test loading of an GPX file from SBN file'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220411_111600.gpx')
        track = Track(filename)
        track.load()

        self.assertEqual(track.fields['ts']['decimals'], 0)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['ele']['decimals'], 2)
        self.assertEqual(track.fields['cog']['decimals'], 2)
        self.assertEqual(track.fields['sog']['decimals'], 2)
        self.assertEqual(track.fields['hdop']['decimals'], 1)


    def testGpxSbp(self):
        '''Test loading of an GPX file from SBP file'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220512_094254_DLG.gpx')
        track = Track(filename)
        track.load()

        self.assertEqual(track.fields['ts']['decimals'], 0)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['ele']['decimals'], 2)
        self.assertEqual(track.fields['cog']['decimals'], 2)
        self.assertEqual(track.fields['sog']['decimals'], 2)
        self.assertEqual(track.fields['hdop']['decimals'], 1)


    def testGpxFenix6(self):
        '''Test loading of an GPX file from Fenix 6'''

        filename = os.path.join(projdir, 'sessions', '20220422', 'activity_8686414511.gpx')
        track = Track(filename)
        track.load()

        self.assertEqual(track.fields['ts']['decimals'], 0)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['ele']['decimals'], 1)


    def testGpxOsMaps(self):
        '''Test loading of an GPX file from OS Maps'''

        filename = os.path.join(projdir, 'sessions', 'misc', 'Seatownandgoldencap.gpx')
        track = Track(filename)
        track.load()

        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)


    def testNmeaGt31(self):
        '''Test loading of an NMEA file from a GT-31'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220411_111600.nmea')
        track = Track(filename)
        track.load()

        self.assertEqual(track.fields['ts']['decimals'], 0)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['sog']['decimals'], 3)
        self.assertEqual(track.fields['cog']['decimals'], 2)
        self.assertEqual(track.fields['hdop']['decimals'], 1)
        self.assertEqual(track.fields['ele']['decimals'], 2)


    def testOaoMini(self):
        '''Test loading of an OAO file from a Motion Mini'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'Motion_Mini_10Hz_0470_2022-04-11-1117.oao')
        track = Track(filename)
        track.load()

        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['ele']['decimals'], 3)
        self.assertEqual(track.fields['sog']['decimals'], 3)
        self.assertEqual(track.fields['cog']['decimals'], 3)
        self.assertEqual(track.fields['ts']['decimals'], 1)
        self.assertEqual(track.fields['sacc']['decimals'], 3)
        self.assertEqual(track.fields['hacc']['decimals'], 3)
        self.assertEqual(track.fields['vacc']['decimals'], 3)
        self.assertEqual(track.fields['cacc']['decimals'], 3)
        self.assertEqual(track.fields['hdop']['decimals'], 2)


    def testOaoEsp(self):
        '''Test loading of an OAO file from a ESP-GPS'''

        filename = os.path.join(projdir, 'sessions', '20211230-esp', 'Head_L_7C9EBDFAF5C8_007.oao')
        track = Track(filename)
        track.load()

        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['ele']['decimals'], 3)
        self.assertEqual(track.fields['sog']['decimals'], 3)
        self.assertEqual(track.fields['cog']['decimals'], 3)
        self.assertEqual(track.fields['ts']['decimals'], 3)
        self.assertEqual(track.fields['sacc']['decimals'], 3)
        self.assertEqual(track.fields['hacc']['decimals'], 3)
        self.assertEqual(track.fields['vacc']['decimals'], 3)
        self.assertEqual(track.fields['cacc']['decimals'], 3)
        self.assertEqual(track.fields['hdop']['decimals'], 2)


    def testSbnGt11(self):
        '''Test loading of an SBN file from a GT-11'''

        filename = os.path.join(projdir, 'sessions', '20071227', 'MIKE_G_1003053_20071227_133836.SBN')
        track = Track(filename)
        track.load()

        self.assertEqual(track.fields['ts']['decimals'], 3)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['ele']['decimals'], 2)
        self.assertEqual(track.fields['sog']['decimals'], 2)
        self.assertEqual(track.fields['cog']['decimals'], 2)
        self.assertEqual(track.fields['roc']['decimals'], 2)
        self.assertEqual(track.fields['ehpe']['decimals'], 2)
        self.assertEqual(track.fields['evpe']['decimals'], 2)
        self.assertEqual(track.fields['hdop']['decimals'], 1)
        self.assertEqual(track.fields['sogu']['decimals'], 2)
        self.assertEqual(track.fields['cogu']['decimals'], 2)


    def testSbnGt31(self):
        '''Test loading of an SBN file from a GT-31'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220411_111600.SBN')
        track = Track(filename)
        track.load()

        self.assertEqual(track.fields['ts']['decimals'], 0)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['ele']['decimals'], 2)
        self.assertEqual(track.fields['sog']['decimals'], 2)
        self.assertEqual(track.fields['cog']['decimals'], 2)
        self.assertEqual(track.fields['roc']['decimals'], 2)
        self.assertEqual(track.fields['ehpe']['decimals'], 2)
        self.assertEqual(track.fields['evpe']['decimals'], 2)
        self.assertEqual(track.fields['hdop']['decimals'], 1)
        self.assertEqual(track.fields['sogu']['decimals'], 2)
        self.assertEqual(track.fields['cogu']['decimals'], 2)
        self.assertEqual(track.fields['sdop']['decimals'], 2)
        self.assertEqual(track.fields['vsdop']['decimals'], 2)


    def testSbpGt11(self):
        '''Test loading of an SBP file from a GT-11'''

        filename = os.path.join(projdir, 'sessions', '20071227', 'MIKE_G_1003053_20071227_165512_DLG.SBP')
        track = Track(filename)
        track.load()

        self.assertEqual(track.fields['ts']['decimals'], 3)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['ele']['decimals'], 2)
        self.assertEqual(track.fields['sog']['decimals'], 2)
        self.assertEqual(track.fields['cog']['decimals'], 2)
        self.assertEqual(track.fields['roc']['decimals'], 2)
        self.assertEqual(track.fields['hdop']['decimals'], 1)


    def testSbpGt31(self):
        '''Test loading of an SBP file from a GT-31'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220512_094254_DLG.SBP')
        track = Track(filename)
        track.load()

        self.assertEqual(track.fields['ts']['decimals'], 0)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['ele']['decimals'], 2)
        self.assertEqual(track.fields['sog']['decimals'], 2)
        self.assertEqual(track.fields['cog']['decimals'], 2)
        self.assertEqual(track.fields['roc']['decimals'], 2)
        self.assertEqual(track.fields['hdop']['decimals'], 1)
        self.assertEqual(track.fields['sdop']['decimals'], 2)
        self.assertEqual(track.fields['vsdop']['decimals'], 2)


    def testUbxEsp(self):
        '''Test loading of a UBX file from an ESP-GPS'''

        filename = os.path.join(projdir, 'sessions', '20211230-esp', 'Head_L_7C9EBDFAF5C8_007.ubx')
        track = Track(filename)
        track.load()

        self.assertEqual(track.fields['ts']['decimals'], 3)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['ele']['decimals'], 3)
        self.assertEqual(track.fields['hacc']['decimals'], 3)
        self.assertEqual(track.fields['vacc']['decimals'], 3)
        self.assertEqual(track.fields['roc']['decimals'], 3)
        self.assertEqual(track.fields['sog']['decimals'], 3)
        self.assertEqual(track.fields['cog']['decimals'], 3)
        self.assertEqual(track.fields['sacc']['decimals'], 3)
        self.assertEqual(track.fields['cacc']['decimals'], 3)
        self.assertEqual(track.fields['pdop']['decimals'], 2)

In [4]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

..............
----------------------------------------------------------------------
Ran 14 tests in 3.539s

OK
